In [358]:
# %pip install -q keras-nightly
# %pip install scipy
# %pip install scikit-learn

In [359]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [360]:
fileName = "isFollow"
mangas_df = pd.read_csv('../Collaborative Filtering/mangas.csv')
behaviors_df = pd.read_csv(f'../Collaborative Filtering/behaviors_{fileName}.csv')

In [361]:
if fileName == "isFollow":
    behaviors_df['isFollow'] = behaviors_df['isFollow'].replace({True: 1, False: 0}).infer_objects(copy=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17516\505009233.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  behaviors_df['isFollow'] = behaviors_df['isFollow'].replace({True: 1, False: 0}).infer_objects(copy=False)


In [362]:
behaviors_df[fileName].fillna(0, inplace=True)
user_id_list = behaviors_df['userId'].unique()
manga_id_list = mangas_df['_id'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_17516\2389881273.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  behaviors_df[fileName].fillna(0, inplace=True)


In [363]:
expanded_data = [{'userId': user_id, '_id': manga_id} for user_id in user_id_list for manga_id in manga_id_list]
expanded_df = pd.DataFrame(expanded_data)
expanded_df.rename(columns={'_id': 'mangaId'}, inplace=True)

In [364]:
expanded_df

,userId,mangaId
0,664e4162e53e1e7c3e8c5cec,664f7bbff7d334bb0ba94e00
1,664e4162e53e1e7c3e8c5cec,66648d1fe14004ab2684b308
2,664e4162e53e1e7c3e8c5cec,66648eb0e14004ab2684b3c3
3,664e4162e53e1e7c3e8c5cec,66648f04e14004ab2684b3e4
4,664e4162e53e1e7c3e8c5cec,66649528e14004ab2684b790
...,...,...
283,667c09d0f39ca48723d291cf,66787ec307cebf7337496017
284,667c09d0f39ca48723d291cf,66787fc607cebf7337496095
285,667c09d0f39ca48723d291cf,6678806007cebf73374960c3
286,667c09d0f39ca48723d291cf,6678836f07cebf7337496332


In [365]:
merged_df = pd.merge(expanded_df, behaviors_df, on=['userId', 'mangaId'], how='left')
merged_df[fileName] = merged_df[fileName].fillna(0)
merged_df.drop(columns=['updatedAt'], inplace=True)

In [366]:
merged_df

,userId,mangaId,isFollow
0,664e4162e53e1e7c3e8c5cec,664f7bbff7d334bb0ba94e00,1.0
1,664e4162e53e1e7c3e8c5cec,66648d1fe14004ab2684b308,1.0
2,664e4162e53e1e7c3e8c5cec,66648eb0e14004ab2684b3c3,0.0
3,664e4162e53e1e7c3e8c5cec,66648f04e14004ab2684b3e4,1.0
4,664e4162e53e1e7c3e8c5cec,66649528e14004ab2684b790,1.0
...,...,...,...
283,667c09d0f39ca48723d291cf,66787ec307cebf7337496017,0.0
284,667c09d0f39ca48723d291cf,66787fc607cebf7337496095,0.0
285,667c09d0f39ca48723d291cf,6678806007cebf73374960c3,0.0
286,667c09d0f39ca48723d291cf,6678836f07cebf7337496332,0.0


In [367]:
mangas_df.rename(columns={'_id': 'mangaId'}, inplace=True)
datas = pd.merge(mangas_df, merged_df, on='mangaId').drop(['author','genre'],axis=1)

userDatas = datas.pivot_table(index=['userId'],columns=['mangaId'],values=fileName)

In [368]:
userDatas

mangaId,664f7bbff7d334bb0ba94e00,66648d1fe14004ab2684b308,66648eb0e14004ab2684b3c3,66648f04e14004ab2684b3e4,66649528e14004ab2684b790,666495a0e14004ab2684b890,66649682e14004ab2684b90c,667098f93ceb1e17489672d0,667868b8c070b2f1253fb148,6678698fc070b2f1253fb1d2,...,6678783207cebf7337495d20,6678797c07cebf7337495ef1,66787a1407cebf7337495f25,66787ace07cebf7337495f6d,66787e4607cebf7337495fe3,66787ec307cebf7337496017,66787fc607cebf7337496095,6678806007cebf73374960c3,6678836f07cebf7337496332,667883f207cebf7337496366
userId,,,,,,,,,,,,,,,,,,,,,
664e4162e53e1e7c3e8c5cec,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
664f80d9f7d334bb0ba95156,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
665b51dfbfe859d2126fd77a,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
667c0844f39ca48723d290c8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
667c086cf39ca48723d290e9,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
667c0893f39ca48723d29104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
667c08bbf39ca48723d29126,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
667c08f5f39ca48723d29147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
667c0918f39ca48723d29169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [369]:
def standardize(row):
    return (row - row.mean()) / (row.max() - row.min())
df_std = userDatas.apply(standardize).T.fillna(0)

In [370]:
# df_2x2 = df_std.loc[['664f7bbff7d334bb0ba94e00', '66648d1fe14004ab2684b308', '66648eb0e14004ab2684b3c3'], ['664e4162e53e1e7c3e8c5cec', '664f80d9f7d334bb0ba95156', '665b51dfbfe859d2126fd77a']]
# df_2x2

In [371]:
sparse_df = sparse.csr_matrix(df_std.values)
cosine_sim_matrix = pd.DataFrame(cosine_similarity(sparse_df), index=userDatas.columns, columns=userDatas.columns)



In [372]:
cosine_sim_matrix

mangaId,664f7bbff7d334bb0ba94e00,66648d1fe14004ab2684b308,66648eb0e14004ab2684b3c3,66648f04e14004ab2684b3e4,66649528e14004ab2684b790,666495a0e14004ab2684b890,66649682e14004ab2684b90c,667098f93ceb1e17489672d0,667868b8c070b2f1253fb148,6678698fc070b2f1253fb1d2,...,6678783207cebf7337495d20,6678797c07cebf7337495ef1,66787a1407cebf7337495f25,66787ace07cebf7337495f6d,66787e4607cebf7337495fe3,66787ec307cebf7337496017,66787fc607cebf7337496095,6678806007cebf73374960c3,6678836f07cebf7337496332,667883f207cebf7337496366
mangaId,,,,,,,,,,,,,,,,,,,,,
664f7bbff7d334bb0ba94e00,1.000000e+00,0.597614,0.478091,8.164966e-01,6.250000e-01,8.164966e-01,0.836660,1.250000e-01,0.158114,0.426401,...,0.632456,-1.250000e-01,-0.316228,0.158114,-1.387779e-17,4.082483e-01,-1.250000e-01,-1.250000e-01,0.632456,-0.239046
66648d1fe14004ab2684b308,5.976143e-01,1.000000,0.714286,4.879500e-01,2.390457e-01,4.879500e-01,0.371429,1.195229e-01,0.377964,0.254824,...,0.377964,-1.195229e-01,-0.075593,-0.075593,9.759001e-02,9.759001e-02,-1.195229e-01,5.976143e-01,0.377964,0.028571
66648eb0e14004ab2684b3c3,4.780914e-01,0.714286,1.000000,2.927700e-01,1.195229e-01,2.927700e-01,0.314286,-1.195229e-01,0.075593,0.356753,...,0.075593,-2.390457e-01,-0.377964,0.075593,2.927700e-01,-9.759001e-02,-2.390457e-01,4.780914e-01,0.529150,-0.028571
66648f04e14004ab2684b3e4,8.164966e-01,0.487950,0.292770,1.000000e+00,4.082483e-01,5.555556e-01,0.683130,2.775558e-17,0.258199,0.522233,...,0.774597,-1.387779e-17,-0.258199,0.258199,-3.333333e-01,5.555556e-01,-1.387779e-17,-2.775558e-17,0.258199,-0.097590
66649528e14004ab2684b790,6.250000e-01,0.239046,0.119523,4.082483e-01,1.000000e+00,8.164966e-01,0.836660,1.250000e-01,0.158114,0.426401,...,0.632456,-1.250000e-01,-0.316228,0.158114,1.387779e-17,8.164966e-01,2.500000e-01,-1.250000e-01,0.632456,-0.239046
666495a0e14004ab2684b890,8.164966e-01,0.487950,0.292770,5.555556e-01,8.164966e-01,1.000000e+00,0.683130,4.082483e-01,0.258199,0.522233,...,0.774597,-4.163336e-17,-0.258199,0.258199,1.111111e-01,5.555556e-01,-4.163336e-17,-2.775558e-17,0.774597,-0.097590
66649682e14004ab2684b90c,8.366600e-01,0.371429,0.314286,6.831301e-01,8.366600e-01,6.831301e-01,1.000000,-1.195229e-01,0.075593,0.356753,...,0.529150,-2.390457e-01,-0.377964,0.075593,-9.759001e-02,6.831301e-01,1.195229e-01,-2.390457e-01,0.529150,-0.371429
667098f93ceb1e17489672d0,1.250000e-01,0.119523,-0.119523,2.775558e-17,1.250000e-01,4.082483e-01,-0.119523,1.000000e+00,0.316228,0.213201,...,0.316228,1.250000e-01,0.316228,0.316228,4.082483e-01,2.775558e-17,1.250000e-01,1.250000e-01,0.316228,0.239046
667868b8c070b2f1253fb148,1.581139e-01,0.377964,0.075593,2.581989e-01,1.581139e-01,2.581989e-01,0.075593,3.162278e-01,1.000000,0.674200,...,0.400000,6.324555e-01,0.400000,0.400000,-2.581989e-01,2.581989e-01,6.324555e-01,6.324555e-01,0.400000,0.529150


In [373]:
corrMatrix = userDatas.corr(method='pearson')

In [374]:
corrMatrix

mangaId,664f7bbff7d334bb0ba94e00,66648d1fe14004ab2684b308,66648eb0e14004ab2684b3c3,66648f04e14004ab2684b3e4,66649528e14004ab2684b790,666495a0e14004ab2684b890,66649682e14004ab2684b90c,667098f93ceb1e17489672d0,667868b8c070b2f1253fb148,6678698fc070b2f1253fb1d2,...,6678783207cebf7337495d20,6678797c07cebf7337495ef1,66787a1407cebf7337495f25,66787ace07cebf7337495f6d,66787e4607cebf7337495fe3,66787ec307cebf7337496017,66787fc607cebf7337496095,6678806007cebf73374960c3,6678836f07cebf7337496332,667883f207cebf7337496366
mangaId,,,,,,,,,,,,,,,,,,,,,
664f7bbff7d334bb0ba94e00,1.000000e+00,0.597614,0.478091,8.164966e-01,6.250000e-01,8.164966e-01,0.836660,1.250000e-01,0.158114,0.426401,...,0.632456,-1.250000e-01,-0.316228,0.158114,6.798700e-17,4.082483e-01,-1.250000e-01,-1.250000e-01,0.632456,-0.239046
66648d1fe14004ab2684b308,5.976143e-01,1.000000,0.714286,4.879500e-01,2.390457e-01,4.879500e-01,0.371429,1.195229e-01,0.377964,0.254824,...,0.377964,-1.195229e-01,-0.075593,-0.075593,9.759001e-02,9.759001e-02,-1.195229e-01,5.976143e-01,0.377964,0.028571
66648eb0e14004ab2684b3c3,4.780914e-01,0.714286,1.000000,2.927700e-01,1.195229e-01,2.927700e-01,0.314286,-1.195229e-01,0.075593,0.356753,...,0.075593,-2.390457e-01,-0.377964,0.075593,2.927700e-01,-9.759001e-02,-2.390457e-01,4.780914e-01,0.529150,-0.028571
66648f04e14004ab2684b3e4,8.164966e-01,0.487950,0.292770,1.000000e+00,4.082483e-01,5.555556e-01,0.683130,-1.133117e-17,0.258199,0.522233,...,0.774597,-6.232141e-17,-0.258199,0.258199,-3.333333e-01,5.555556e-01,-2.832792e-17,-5.665583e-18,0.258199,-0.097590
66649528e14004ab2684b790,6.250000e-01,0.239046,0.119523,4.082483e-01,1.000000e+00,8.164966e-01,0.836660,1.250000e-01,0.158114,0.426401,...,0.632456,-1.250000e-01,-0.316228,0.158114,4.532467e-17,8.164966e-01,2.500000e-01,-1.250000e-01,0.632456,-0.239046
666495a0e14004ab2684b890,8.164966e-01,0.487950,0.292770,5.555556e-01,8.164966e-01,1.000000e+00,0.683130,4.082483e-01,0.258199,0.522233,...,0.774597,-6.232141e-17,-0.258199,0.258199,1.111111e-01,5.555556e-01,-2.832792e-17,-7.365258e-17,0.774597,-0.097590
66649682e14004ab2684b90c,8.366600e-01,0.371429,0.314286,6.831301e-01,8.366600e-01,6.831301e-01,1.000000,-1.195229e-01,0.075593,0.356753,...,0.529150,-2.390457e-01,-0.377964,0.075593,-9.759001e-02,6.831301e-01,1.195229e-01,-2.390457e-01,0.529150,-0.371429
667098f93ceb1e17489672d0,1.250000e-01,0.119523,-0.119523,-1.133117e-17,1.250000e-01,4.082483e-01,-0.119523,1.000000e+00,0.316228,0.213201,...,0.316228,1.250000e-01,0.316228,0.316228,4.082483e-01,5.099025e-17,1.250000e-01,1.250000e-01,0.316228,0.239046
667868b8c070b2f1253fb148,1.581139e-01,0.377964,0.075593,2.581989e-01,1.581139e-01,2.581989e-01,0.075593,3.162278e-01,1.000000,0.674200,...,0.400000,6.324555e-01,0.400000,0.400000,-2.581989e-01,2.581989e-01,6.324555e-01,6.324555e-01,0.400000,0.529150


In [375]:
def get_recommendations(user_id, userViews, corrMatrix, num_recommendations=36):
    user_data = userViews.loc[user_id]
    watched_items = user_data[user_data > 0].index.tolist()
    
    recommendations = {}
    for item in watched_items:
        similar_items = corrMatrix[item].drop(item).sort_values(ascending=False)
        for similar_item, score in similar_items.items():
            if np.isnan(score):  # Kiểm tra và loại bỏ giá trị NaN
                continue
            if similar_item in recommendations:
                recommendations[similar_item] += score
            else:
                recommendations[similar_item] = score

    for item in watched_items:
        if item in recommendations:
            recommendations[item] += 1.0
        else:
            recommendations[item] = 1.0 

    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return sorted_recommendations[:num_recommendations]

In [376]:
user_id = '667c08bbf39ca48723d29126'
# recommendations = get_recommendations(user_id, userDatas, corrMatrix)
recommendations = get_recommendations(user_id, userDatas, cosine_sim_matrix)
recommendations


[('667875ef07cebf7337495a38', 4.728862355862756),
 ('664f7bbff7d334bb0ba94e00', 4.728862355862756),
 ('66648f04e14004ab2684b3e4', 4.096843271278251),
 ('66649682e14004ab2684b90c', 4.04216438984641),
 ('66648d1fe14004ab2684b308', 3.768892931522945),
 ('666495a0e14004ab2684b890', 3.6523988268338075),
 ('66648eb0e14004ab2684b3c3', 3.2775243379235035),
 ('6678783207cebf7337495d20', 3.022215363132826),
 ('6678836f07cebf7337496332', 2.9593749512495764),
 ('66649528e14004ab2684b790', 2.8534768997982556),
 ('66786bccc070b2f1253fb266', 2.8534768997982556),
 ('6678698fc070b2f1253fb1d2', 2.3433654970210243),
 ('66787ec307cebf7337496017', 2.0551821875472545),
 ('66786c78c070b2f1253fb2ad', 1.6328243081714677),
 ('667868b8c070b2f1253fb148', 1.1035769179769173),
 ('66787ace07cebf7337495f6d', 0.6500195503658445),
 ('6678806007cebf73374960c3', 0.5866600265340752),
 ('667098f93ceb1e17489672d0', 0.1304771390665609),
 ('66787e4607cebf7337495fe3', -0.040563311448773204),
 ('66787fc607cebf7337496095', -0.48

In [377]:
filtered_recommendations = [item_id for item_id, score in recommendations if not np.isnan(score)]
filtered_recommendations

['667875ef07cebf7337495a38',
 '664f7bbff7d334bb0ba94e00',
 '66648f04e14004ab2684b3e4',
 '66649682e14004ab2684b90c',
 '66648d1fe14004ab2684b308',
 '666495a0e14004ab2684b890',
 '66648eb0e14004ab2684b3c3',
 '6678783207cebf7337495d20',
 '6678836f07cebf7337496332',
 '66649528e14004ab2684b790',
 '66786bccc070b2f1253fb266',
 '6678698fc070b2f1253fb1d2',
 '66787ec307cebf7337496017',
 '66786c78c070b2f1253fb2ad',
 '667868b8c070b2f1253fb148',
 '66787ace07cebf7337495f6d',
 '6678806007cebf73374960c3',
 '667098f93ceb1e17489672d0',
 '66787e4607cebf7337495fe3',
 '66787fc607cebf7337496095',
 '667873223f6da702146853d2',
 '6678797c07cebf7337495ef1',
 '667883f207cebf7337496366',
 '66787a1407cebf7337495f25']